In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import find_peaks
from scipy.fft import fft, fftfreq

In [2]:
def read_and_prepocess(path):
    data = pd.read_csv(path, header = None, names = ['x', 'y', 'p', 't'])
    data['t'] = data['t'] - data['t'].min()
    data['t'] = data['t']/1000000
    return data

def find_most_frequent_pairs(data):
    counts = data.groupby(['x','y']).size().reset_index(name='counts')
    most_common_pair = counts.sort_values('counts', ascending=False)
    return most_common_pair

def pixel_stats_most_common(data,position,most_common_pair):
    df_pixel = data[(data['x']==most_common_pair.iloc[position]['x']) & (data['y']==most_common_pair.iloc[position]['y'])].sort_values('t')
    return df_pixel

def pixel_specific(data,x,y):
    df_pixel = data[(data['x']==x) & (data['y']==y)].sort_values('t')
    return df_pixel

def plot_pixel_value(data, samples, step):
    data[0:samples].plot(x = 't', y = 'p', kind = 'line', marker = '*',  figsize = (25,15), xticks = np.arange(data['t'].min(), data['t'].max(),1))


def plot_fft(pixel, xlim, ylim):

    time = pixel['t']
    amplitude = pixel['p']

    # Calculate the Fast Fourier Transform (FFT) of the signal
    fft_result = fft(amplitude)
    amplitude_spectrum = np.abs(fft_result)

    # Calculate the frequencies corresponding to the FFT
    frequencies = fftfreq(len(fft_result), d = 1)

    # Plot the amplitude spectrum (optional)
    plt.scatter(frequencies, amplitude_spectrum)
    plt.ylim(0,ylim)
    plt.xlim(0,xlim)
    plt.title('Amplitude Spectrum')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.show()
    

def plot_fft_mulitple(data, xlim, ylim,number):

    mfp = find_most_frequent_pairs(data)
    for i in range(0, number):
        pixel = pixel_specific(data, mfp['x'].iloc[i], mfp['y'].iloc[i])
        plot_fft(pixel,xlim,ylim)
        print(mfp['x'].iloc[i])
        print(mfp['y'].iloc[i])
    
        
def plot_most_common(data, most_common, num_of_plots, samples):
    
    for i in range(0,num_of_plots):
        x = most_common['x'].iloc[i]
        y = most_common['y'].iloc[i]
        pixel = pixel_specific(data,x,y)
        plt.plot(pixel['t'][:samples], pixel['p'][:samples], marker='*', )
        plt.show()
        print(x)
        print(y)



def plot_TimeAndFrequency(data, xlim, ylim, numOfPlots, samples = 50):
    mfp = find_most_frequent_pairs(data)
    for i in range(0, numOfPlots):
        x_coord = mfp['x'].iloc[i]
        y_coord = mfp['y'].iloc[i]
        pixel = pixel_specific(data, x_coord, y_coord)
        time = pixel['t']
        amplitude = pixel['p']

        # Calculate the Fast Fourier Transform (FFT) of the signal
        fft_result = fft(amplitude)
        amplitude_spectrum = np.abs(fft_result)
        # Calculate the frequencies corresponding to the FFT
        frequencies = fftfreq(len(fft_result), d = 1)

        fig, axs = plt.subplots(1, 2, figsize=(15, 6))

        axs[0].plot(pixel['t'][:samples], pixel['p'][:samples], marker='*', )
        titleSpatial = f'Time Domain plot of Pixel values : ({x_coord},{y_coord})'
        axs[0].set_title(titleSpatial)


        axs[1].scatter(frequencies, amplitude_spectrum)
        axs[1].set_ylim(0,ylim)
        axs[1].set_xlim(0,xlim)
        titleFFT = f'FFT plot of Pixel values: ({x_coord},{y_coord})'
        axs[1].set_title(titleFFT)
        axs[1].set_xlabel('Frequency (Hz)')
        axs[1].set_ylabel('Amplitude')

        plt.tight_layout()
        plt.show()
    

